In [1]:
import pandas as pd
from pathlib import Path
import logging
from box import Box
from datetime import datetime
import sys
import torch

In [2]:
# !pip uninstall fastai

In [5]:
from fast_bert import BertLearner, BertDataBunch, accuracy

In [6]:
pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

In [7]:
PATH = Path("../sample_data/imdb_movie_reviews")
DATA_PATH = PATH/'data'
LABEL_PATH = PATH/'label'
OUT_PATH = PATH/'.output'
OUT_PATH.mkdir(exist_ok=True)

MODEL_PATH=OUT_PATH/'model'
MODEL_PATH.mkdir(exist_ok=True)

LOG_PATH = OUT_PATH/'logs/'
LOG_PATH.mkdir(exist_ok=True)

In [8]:
args = Box({
    "run_text": "ibdm_reviews",
    "max_seq_length": 512,
    "batch_size": 8,
    "learning_rate": 5e-5,
    "num_train_epochs": 6,
    "fp16": False,
    "model_name": 'albert-base-v2',
    "model_type": 'albert'
})

device = torch.device('cuda') if torch.cuda.device_count() else torch.device('cpu')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

In [9]:
logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [10]:
device

device(type='cpu')

In [11]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, args.model_name, 
                          train_file="train_sample.csv", val_file="val_sample.csv",
                          batch_size_per_gpu=args.batch_size, 
                          max_seq_length=args.max_seq_length, 
                          multi_gpu=args.multi_gpu,
                          multi_label=False,
                          model_type=args.model_type
                         )

12/14/2019 08:19:40 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-spiece.model from cache at /Users/kaushaltrivedi/.cache/torch/transformers/dd1588b85b6fdce1320e224d29ad062e97588e17326b9d05a0b29ee84b8f5f93.c81d4deb77aec08ce575b7a39a989a79dd54f321bfb82c2b54dd35f52f8182cf
12/14/2019 08:19:40 - INFO - root -   Loading features from cached file ../sample_data/imdb_movie_reviews/data/cache/cached_albert_train_multi_class_512_train_sample.csv
12/14/2019 08:19:40 - INFO - root -   Loading features from cached file ../sample_data/imdb_movie_reviews/data/cache/cached_albert_dev_multi_class_512_val_sample.csv


In [12]:
metrics = [{"name": "accuracy", "function": accuracy}]

In [13]:
learner = BertLearner.from_pretrained_model(databunch, args.model_name, metrics=metrics, 
                                            device=device, multi_gpu=args.multi_gpu, is_fp16=args.fp16,
                                            multi_label=False, logging_steps=0,
                                            output_dir=OUT_PATH, logger=logger
                                           )

12/14/2019 08:19:41 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-config.json from cache at /Users/kaushaltrivedi/.cache/torch/transformers/0bbb1531ce82f042a813219ffeed7a1fa1f44cd8f78a652c47fc5311e0d40231.ea38adf566af403b92e06f53ac1b443ebd324162fc74a90de95609963d429505
12/14/2019 08:19:41 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "finetuning_task": null,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "num_memory_blocks": 0,
  "output_attentions": fa

In [14]:
learner.fit(2, args.learning_rate, validate=True)

../sample_data/imdb_movie_reviews/.output/tensorboard
12/14/2019 08:19:43 - INFO - root -   ***** Running training *****
12/14/2019 08:19:43 - INFO - root -     Num examples = 100
12/14/2019 08:19:43 - INFO - root -     Num Epochs = 2
12/14/2019 08:19:43 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
12/14/2019 08:19:43 - INFO - root -     Gradient Accumulation steps = 1
12/14/2019 08:19:43 - INFO - root -     Total optimization steps = 26


12/14/2019 08:25:30 - INFO - root -   Running evaluation
12/14/2019 08:25:30 - INFO - root -     Num examples = 50
12/14/2019 08:25:30 - INFO - root -     Batch size = 16


12/14/2019 08:26:31 - INFO - root -   eval_loss after epoch 1: 0.8099662363529205: 
12/14/2019 08:26:31 - INFO - root -   eval_accuracy after epoch 1: 0.36: 
12/14/2019 08:26:31 - INFO - root -   lr after epoch 1: 2.5e-05
12/14/2019 08:26:31 - INFO - root -   train_loss after epoch 1: 0.7384268320523776
12/14/2019 08:26:31 - INFO - root -   

12/14/2019 08:32:17 - INFO - root -   Running evaluation
12/14/2019 08:32:17 - INFO - root -     Num examples = 50
12/14/2019 08:32:17 - INFO - root -     Batch size = 16


12/14/2019 08:33:16 - INFO - root -   eval_loss after epoch 2: 0.8012044578790665: 
12/14/2019 08:33:16 - INFO - root -   eval_accuracy after epoch 2: 0.36: 
12/14/2019 08:33:16 - INFO - root -   lr after epoch 2: 0.0
12/14/2019 08:33:16 - INFO - root -   train_loss after epoch 2: 0.7632057254131024
12/14/2019 08:33:16 - INFO - root -   



(26, 0.7508162787327399)

In [15]:
learner.save_model()

12/14/2019 08:33:16 - INFO - transformers.configuration_utils -   Configuration saved in ../sample_data/imdb_movie_reviews/.output/model_out/config.json
12/14/2019 08:33:16 - INFO - transformers.modeling_utils -   Model weights saved in ../sample_data/imdb_movie_reviews/.output/model_out/pytorch_model.bin


In [16]:
from fast_bert.prediction import BertClassificationPredictor

In [17]:
predictor = BertClassificationPredictor(OUT_PATH/'model_out', LABEL_PATH, multi_label=False, model_type=args.model_type)

12/14/2019 08:51:34 - INFO - transformers.tokenization_utils -   Model name '../sample_data/imdb_movie_reviews/.output/model_out' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming '../sample_data/imdb_movie_reviews/.output/model_out' is a path or url to a directory containing tokenizer files.
12/14/2019 08:51:34 - INFO - transformers.tokenization_utils -   loading file ../sample_data/imdb_movie_reviews/.output/model_out/spiece.model
12/14/2019 08:51:34 - INFO - transformers.tokenization_utils -   loading file ../sample_data/imdb_movie_reviews/.output/model_out/added_tokens.json
12/14/2019 08:51:34 - INFO - transformers.tokenization_utils -   loading file ../sample_data/imdb_movie_reviews/.output/model_out/special_tokens_map.json
12/14/2019 08:51:34 - INFO - transformers.tokenization_utils -   loading file ../sample_data/imdb_movie_reviews/.output/mo

In [20]:
predictor.predict_batch(["i like you"])

12/14/2019 12:52:36 - INFO - root -   Writing example 0 of 1


[[('1', 0.6604397296905518), ('0', 0.33956027030944824)]]